<a href="https://colab.research.google.com/github/subhrajyoti-ghosh/ML-and-Deep-Learning/blob/main/Gesture_Recognition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gesture Recognition
In this group project, you are going to build a 3D Conv model that will be able to predict the 5 gestures correctly. Please import the following libraries to get started.

In [1]:
import numpy as np
import os
#from scipy.misc import imresize
from imageio import imread
import cv2
import datetime
import os

We set the random seed so that the results don't vary drastically.

In [2]:
np.random.seed(30)
import random as rn
rn.seed(30)
from keras import backend as K
import tensorflow as tf
tf.random.set_seed(30)

In this block, you read the folder names for training and validation. You also set the `batch_size` here. Note that you set the batch size in such a way that you are able to use the GPU in full capacity. You keep increasing the batch size until the machine throws an error.

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
train_doc = np.random.permutation(open('/content/drive/MyDrive/Colab Notebooks/Gesture_Recognition/train.csv').readlines())
val_doc = np.random.permutation(open('/content/drive/MyDrive/Colab Notebooks/Gesture_Recognition/val.csv').readlines())
batch_size = 512 #experiment with the batch size

## Generator
This is one of the most important part of the code. The overall structure of the generator has been given. In the generator, you are going to preprocess the images as you have images of 2 different dimensions as well as create a batch of video frames. You have to experiment with `img_idx`, `y`,`z` and normalization such that you get high accuracy.

In [5]:
def generator(source_path, folder_list, batch_size):
    y = 50
    z = 50  
    print( 'Source path = ', source_path, '; batch size =', batch_size)
    img_idx = [0,3,7,8]#create a list of image numbers you want to use for a particular video
    while True:
        t = np.random.permutation(folder_list)
        num_batches = len(folder_list)//batch_size # calculate the number of batches
        for batch in range(num_batches-1): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,5,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    print("Batch 1 : ",source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item])
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    batch_data[folder,idx,:,:,0] = tf.image.per_image_standardization(cv2.resize(image,(y,z)))[:,:,0] #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = tf.image.per_image_standardization(cv2.resize(image,(y,z)))[:,:,1] #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = tf.image.per_image_standardization(cv2.resize(image,(y,z)))[:,:,2] #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do

        
        # write the code for the remaining data points which are left after full batches
        num_batches = len(folder_list)%batch_size # calculate the number of batches
        for batch in range(num_batches-1): # we iterate over the number of batches
            batch_data = np.zeros((batch_size,5,y,z,3)) # x is the number of images you use for each video, (y,z) is the final size of the input images and 3 is the number of channels RGB
            batch_labels = np.zeros((batch_size,5)) # batch_labels is the one hot representation of the output
            for folder in range(batch_size): # iterate over the batch_size
                imgs = os.listdir(source_path+'/'+ t[folder + (batch*batch_size)].split(';')[0]) # read all the images in the folder
                for idx,item in enumerate(img_idx): #  Iterate iver the frames/images of a folder to read them in
                    image = imread(source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item]).astype(np.float32)
                    
                    print("Batch 2 : ",source_path+'/'+ t[folder + (batch*batch_size)].strip().split(';')[0]+'/'+imgs[item])
                    #crop the images and resize them. Note that the images are of 2 different shape 
                    #and the conv3D will throw error if the inputs in a batch have different shapes
                    
                    batch_data[folder,idx,:,:,0] = tf.image.per_image_standardization(cv2.resize(image,(y,z)))[:,:,0] #normalise and feed in the image
                    batch_data[folder,idx,:,:,1] = tf.image.per_image_standardization(cv2.resize(image,(y,z)))[:,:,1] #normalise and feed in the image
                    batch_data[folder,idx,:,:,2] = tf.image.per_image_standardization(cv2.resize(image,(y,z)))[:,:,2] #normalise and feed in the image
                    
                batch_labels[folder, int(t[folder + (batch*batch_size)].strip().split(';')[2])] = 1
            yield batch_data, batch_labels #you yield the batch_data and the batch_labels, remember what does yield do


Note here that a video is represented above in the generator as (number of images, height, width, number of channels). Take this into consideration while creating the model architecture.

In [6]:
curr_dt_time = datetime.datetime.now()
train_path = '/content/drive/MyDrive/Colab Notebooks/Gesture_Recognition/train'
val_path = '/content/drive/MyDrive/Colab Notebooks/Gesture_Recognition/val'
num_train_sequences = len(train_doc)
print('# training sequences =', num_train_sequences)
num_val_sequences = len(val_doc)
print('# validation sequences =', num_val_sequences)
num_epochs = 5 # choose the number of epochs
print ('# epochs =', num_epochs)

# training sequences = 663
# validation sequences = 100
# epochs = 5


## Model
Here you make the model using different functionalities that Keras provides. Remember to use `Conv3D` and `MaxPooling3D` and not `Conv2D` and `Maxpooling2D` for a 3D convolution model. You would want to use `TimeDistributed` while building a Conv2D + RNN model. Also remember that the last layer is the softmax. Design the network in such a way that the model is able to give good accuracy on the least number of parameters so that it can fit in the memory of the webcam.

In [7]:
from keras.models import Sequential, Model
from keras.layers import Dense, GRU, Flatten, TimeDistributed, Flatten, BatchNormalization, Activation
from keras.layers.convolutional import Conv3D, MaxPooling3D
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau
from keras import optimizers
import tensorflow.keras as keras

from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.layers import Flatten,Dense,TimeDistributed,GRU,Conv2D
#write your model here

In [8]:
resnet = ResNet50(include_top=False,weights='imagenet',input_shape=(224,224,3))

94781440/94765736 [==============================] - 2s 0us/step


In [9]:
for layer in resnet.layers[:-10]:
    layer.trainable=False

In [10]:
model = Sequential([resnet,Flatten(),Dense(64,activation='relu'),Dense(5,activation='softmax')])

In [11]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 flatten (Flatten)           (None, 100352)            0         
                                                                 
 dense (Dense)               (None, 64)                6422592   
                                                                 
 dense_1 (Dense)             (None, 5)                 325       
                                                                 
Total params: 30,010,629
Trainable params: 10,888,581
Non-trainable params: 19,122,048
_________________________________________________________________


In [12]:
resnet = ResNet50(include_top=False,weights='imagenet',input_shape=(224,224,3))
cnn =Sequential([resnet])
cnn.add(Conv2D(64,(2,2),strides=(1,1)))
cnn.add(Conv2D(16,(3,3),strides=(1,1)))
cnn.add(Flatten())

In [13]:
print(cnn.summary())

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 conv2d (Conv2D)             (None, 6, 6, 64)          524352    
                                                                 
 conv2d_1 (Conv2D)           (None, 4, 4, 16)          9232      
                                                                 
 flatten_1 (Flatten)         (None, 256)               0         
                                                                 
Total params: 24,121,296
Trainable params: 24,068,176
Non-trainable params: 53,120
_________________________________________________________________
None


In [14]:
model = Sequential()
model.add(TimeDistributed(cnn,input_shape=(20,224,224,3)))
model.add(GRU(16,input_shape=(None,20,256),return_sequences=True))
model.add(GRU(8))
model.add(Dense(5,activation='softmax'))

In [15]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed (TimeDistr  (None, 20, 256)          24121296  
 ibuted)                                                         
                                                                 
 gru (GRU)                   (None, 20, 16)            13152     
                                                                 
 gru_1 (GRU)                 (None, 8)                 624       
                                                                 
 dense_2 (Dense)             (None, 5)                 45        
                                                                 
Total params: 24,135,117
Trainable params: 24,081,997
Non-trainable params: 53,120
_________________________________________________________________


In [16]:
resnet = tf.keras.applications.ResNet50(include_top=False,weights='imagenet',input_shape=(224,224,3))
cnn = tf.keras.Sequential([resnet])
cnn.add(tf.keras.layers.Conv2D(64,(2,2),strides=(1,1)))
cnn.add(tf.keras.layers.Conv2D(16,(3,3),strides=(1,1)))
cnn.add(tf.keras.layers.Flatten())
cnn.add(tf.keras.layers.Dense(64))

In [17]:
model = Sequential()
model.add(TimeDistributed(cnn,input_shape=(20,224,224,3)))
model.add(GRU(16,return_sequences=True))
model.add(GRU(8))
#model.add(Dense(5,activation='softmax'))

Now that you have written the model, the next step is to `compile` the model. When you print the `summary` of the model, you'll see the total number of parameters you have to train.

In [18]:
optimiser = tf.keras.optimizers.SGD(learning_rate=0.01, momentum=0.0, nesterov=False, name='SGD') #write your optimizer
model.compile(optimizer=optimiser, loss=tf.keras.losses.SparseCategoricalCrossentropy(), metrics=['accuracy'])
print (model.summary())

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 time_distributed_1 (TimeDis  (None, 20, 64)           24137744  
 tributed)                                                       
                                                                 
 gru_2 (GRU)                 (None, 20, 16)            3936      
                                                                 
 gru_3 (GRU)                 (None, 8)                 624       
                                                                 
Total params: 24,142,304
Trainable params: 24,089,184
Non-trainable params: 53,120
_________________________________________________________________
None


Let us create the `train_generator` and the `val_generator` which will be used in `.fit_generator`.

In [19]:
train_generator = generator(train_path, train_doc, batch_size)
val_generator = generator(val_path, val_doc, batch_size)

In [20]:
model_name = 'model_init' + '_' + str(curr_dt_time).replace(' ','').replace(':','_') + '/'
    
if not os.path.exists(model_name):
    os.mkdir(model_name)
        
filepath = model_name + 'model-{epoch:05d}-{loss:.5f}-{categorical_accuracy:.5f}-{val_loss:.5f}-{val_categorical_accuracy:.5f}.h5'

checkpoint = ModelCheckpoint(filepath, monitor='val_loss', verbose=1, save_best_only=False, save_weights_only=False, mode='auto', period=1)

LR = [tf.keras.callbacks.EarlyStopping(patience=2),
      tf.keras.callbacks.ModelCheckpoint(filepath=filepath),#'model.{epoch:02d}-{val_loss:.2f}.h5'),
    tf.keras.callbacks.TensorBoard(log_dir='./logs')] # write the REducelronplateau code here
callbacks_list = [checkpoint, LR]

#model.fit_generator(train_generator, epochs=10, callbacks=LR)

The `steps_per_epoch` and `validation_steps` are used by `fit_generator` to decide the number of next() calls it need to make.

In [21]:
if (num_train_sequences%batch_size) == 0:
    steps_per_epoch = int(num_train_sequences/batch_size)
else:
    steps_per_epoch = (num_train_sequences//batch_size) + 1

if (num_val_sequences%batch_size) == 0:
    validation_steps = int(num_val_sequences/batch_size)
else:
    validation_steps = (num_val_sequences//batch_size) + 1

Let us now fit the model. This will start training the model and with the help of the checkpoints, you'll be able to save the model at the end of each epoch.

In [22]:
model.fit_generator(train_generator, steps_per_epoch=steps_per_epoch, epochs=num_epochs, verbose=1, 
                    callbacks=callbacks_list,validation_data=val_generator, 
                    validation_steps=validation_steps,
                    class_weight=None,
                    workers=1,
                    initial_epoch=0)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  


Source path =  /content/drive/MyDrive/Colab Notebooks/Gesture_Recognition/train ; batch size = 512
Batch 2 :  /content/drive/MyDrive/Colab Notebooks/Gesture_Recognition/train/WIN_20180926_17_24_58_Pro_Right_Swipe_new/WIN_20180926_17_24_58_Pro_00051.png
Batch 2 :  /content/drive/MyDrive/Colab Notebooks/Gesture_Recognition/train/WIN_20180926_17_24_58_Pro_Right_Swipe_new/WIN_20180926_17_24_58_Pro_00013.png
Batch 2 :  /content/drive/MyDrive/Colab Notebooks/Gesture_Recognition/train/WIN_20180926_17_24_58_Pro_Right_Swipe_new/WIN_20180926_17_24_58_Pro_00023.png
Batch 2 :  /content/drive/MyDrive/Colab Notebooks/Gesture_Recognition/train/WIN_20180926_17_24_58_Pro_Right_Swipe_new/WIN_20180926_17_24_58_Pro_00017.png
Batch 2 :  /content/drive/MyDrive/Colab Notebooks/Gesture_Recognition/train/WIN_20180926_17_35_17_Pro_Stop_new/WIN_20180926_17_35_17_Pro_00006.png
Batch 2 :  /content/drive/MyDrive/Colab Notebooks/Gesture_Recognition/train/WIN_20180926_17_35_17_Pro_Stop_new/WIN_20180926_17_35_17_Pro_0

/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)


Epoch 1/5


InvalidArgumentError: ignored

In [23]:
score, acc = model.evaluate(generator(train_path, train_doc, batch_size))

Source path =  /content/drive/MyDrive/Colab Notebooks/Gesture_Recognition/train ; batch size = 512
Batch 2 :  /content/drive/MyDrive/Colab Notebooks/Gesture_Recognition/train/WIN_20180925_17_55_45_Pro_Thumbs_Down_new/WIN_20180925_17_55_45_Pro_00018.png
Batch 2 :  /content/drive/MyDrive/Colab Notebooks/Gesture_Recognition/train/WIN_20180925_17_55_45_Pro_Thumbs_Down_new/WIN_20180925_17_55_45_Pro_00024.png
Batch 2 :  /content/drive/MyDrive/Colab Notebooks/Gesture_Recognition/train/WIN_20180925_17_55_45_Pro_Thumbs_Down_new/WIN_20180925_17_55_45_Pro_00022.png
Batch 2 :  /content/drive/MyDrive/Colab Notebooks/Gesture_Recognition/train/WIN_20180925_17_55_45_Pro_Thumbs_Down_new/WIN_20180925_17_55_45_Pro_00040.png
Batch 2 :  /content/drive/MyDrive/Colab Notebooks/Gesture_Recognition/train/WIN_20180907_16_40_59_Pro_Left Swipe_new_Left Swipe_new/WIN_20180907_16_40_59_Pro_00012.png
Batch 2 :  /content/drive/MyDrive/Colab Notebooks/Gesture_Recognition/train/WIN_20180907_16_40_59_Pro_Left Swipe_new_

InvalidArgumentError: ignored

Batch 2 :  /content/drive/MyDrive/Colab Notebooks/Gesture_Recognition/train/WIN_20180925_17_47_19_Pro_Right_Swipe_new/WIN_20180925_17_47_19_Pro_00037.png
Batch 2 :  /content/drive/MyDrive/Colab Notebooks/Gesture_Recognition/train/WIN_20180925_17_47_19_Pro_Right_Swipe_new/WIN_20180925_17_47_19_Pro_00035.png
Batch 2 :  /content/drive/MyDrive/Colab Notebooks/Gesture_Recognition/train/WIN_20180925_17_47_19_Pro_Right_Swipe_new/WIN_20180925_17_47_19_Pro_00057.png
Batch 2 :  /content/drive/MyDrive/Colab Notebooks/Gesture_Recognition/train/WIN_20180925_17_47_19_Pro_Right_Swipe_new/WIN_20180925_17_47_19_Pro_00033.png
Batch 2 :  /content/drive/MyDrive/Colab Notebooks/Gesture_Recognition/train/WIN_20180907_16_49_05_Pro_Left Swipe_new_Left Swipe_new/WIN_20180907_16_49_05_Pro_00003.png
Batch 2 :  /content/drive/MyDrive/Colab Notebooks/Gesture_Recognition/train/WIN_20180907_16_49_05_Pro_Left Swipe_new_Left Swipe_new/WIN_20180907_16_49_05_Pro_00035.png
Batch 2 :  /content/drive/MyDrive/Colab Notebook

In [24]:
score

NameError: ignored